In [1]:
#the best code so far, summary & aggregation dropdowns
import sys
import pandas as pd
from PySide6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QTableView, QHeaderView,
    QLabel, QComboBox, QSpinBox, QPushButton, QFileDialog
)
from PySide6.QtCore import Qt, QAbstractTableModel

# Load the raw data
df = pd.read_excel('Leger Dataset 2024 v2_2.xlsx')
df["obs"] = pd.to_datetime(df['obs'])
df["obs"] = df["obs"].dt.strftime("%Y-%m-%d")

# Constants
columns_per_page = 10  
num_columns = df.shape[1]

class PandasModel(QAbstractTableModel):
    def __init__(self, data):
        super().__init__()
        self._data = data

    def rowCount(self, parent=None):
        return self._data.shape[0]

    def columnCount(self, parent=None):
        return self._data.shape[1]

    def data(self, index, role=Qt.DisplayRole):
        if role == Qt.DisplayRole:
            return str(self._data.iloc[index.row(), index.column()])
        return None

    def headerData(self, section, orientation, role):
        if role == Qt.DisplayRole:
            if orientation == Qt.Horizontal:
                return self._data.columns[section]
            if orientation == Qt.Vertical:
                return str(self._data.index[section])
        return None

    def update_data(self, new_data):
        self.beginResetModel()
        self._data = new_data
        self.endResetModel()

class TwoTablesApp(QWidget):
    def __init__(self):
        super().__init__()
        self.df = df  # Initialize the dataframe as an instance variable
        self.init_ui()

    def init_ui(self):
        layout = QVBoxLayout()

        # Summary type dropdown
        self.summary_type_dropdown = QComboBox(self)
        self.summary_type_dropdown.addItems(["Yearly", "Last 52 Weeks"])
        self.summary_type_dropdown.currentTextChanged.connect(self.update_summary_table)
        self.summary_type_dropdown.setStyleSheet("color: white; background-color: #2b2b2b;")

        # Aggregation type dropdown
        self.aggregation_type_dropdown = QComboBox(self)
        self.aggregation_type_dropdown.addItems(["Sum", "Average"])
        self.aggregation_type_dropdown.currentTextChanged.connect(self.update_summary_table)
        self.aggregation_type_dropdown.setStyleSheet("color: white; background-color: #2b2b2b;")

        # Page selector for raw data table
        self.raw_page_selector = QSpinBox(self)
        self.raw_page_selector.setRange(1, (num_columns + columns_per_page - 1) // columns_per_page)
        self.raw_page_selector.valueChanged.connect(self.update_raw_table)
        self.raw_page_selector.setStyleSheet("color: white; background-color: #2b2b2b;")

        # Page selector for summary data table
        self.summary_page_selector = QSpinBox(self)
        self.summary_page_selector.setRange(1, (num_columns + columns_per_page - 1) // columns_per_page)
        self.summary_page_selector.valueChanged.connect(self.update_summary_table)
        self.summary_page_selector.setStyleSheet("color: white; background-color: #2b2b2b;")

        # Export button
        self.export_button = QPushButton("Export to Excel", self)
        self.export_button.setStyleSheet("color: white; background-color: #2b2b2b;")
        self.export_button.clicked.connect(self.export_to_excel)  

        # Add controls to layout
        control_layout = QHBoxLayout()
        control_layout.addWidget(self.summary_type_dropdown)
        control_layout.addWidget(self.aggregation_type_dropdown)
        control_layout.addWidget(QLabel("Raw Table Page:"))
        control_layout.addWidget(self.raw_page_selector)
        control_layout.addWidget(QLabel("Summary Table Page:"))
        control_layout.addWidget(self.summary_page_selector)
        control_layout.addWidget(self.export_button)
        layout.addLayout(control_layout)

        # Summary data table
        self.summary_data_table = QTableView(self)
        self.summary_data_table.setStyleSheet("color: white;")
        self.summary_data_model = PandasModel(pd.DataFrame())
        self.summary_data_table.setModel(self.summary_data_model)
        self.summary_data_table.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        layout.addWidget(self.create_card("Data Summary", self.summary_data_table))

        # Raw data table
        self.raw_data_table = QTableView(self)
        self.raw_data_table.setStyleSheet("color: white;")
        self.raw_data_model = PandasModel(self.df.iloc[:, :columns_per_page])  # Display initial 10 columns
        self.raw_data_table.setModel(self.raw_data_model)
        self.raw_data_table.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        layout.addWidget(self.create_card("Raw Data", self.raw_data_table))

        self.setLayout(layout)
        self.setWindowTitle("Raw Data and Summary Example")
        self.update_tables()

    def create_card(self, title, table):
        card_layout = QVBoxLayout()
        title_label = QLabel(title)
        title_label.setStyleSheet("color: white;")
        card_layout.addWidget(title_label)
        card_layout.addWidget(table)
        card_widget = QWidget()
        card_widget.setLayout(card_layout)
        return card_widget

    def update_tables(self):
        self.update_summary_table()
        self.update_raw_table()

    def update_summary_table(self):
        df = self.df.copy()
        summary_type = self.summary_type_dropdown.currentText().lower()
        aggregation_type = self.aggregation_type_dropdown.currentText().lower()
        selected_page = self.summary_page_selector.value()

        if summary_type == 'yearly':
            df['Period'] = pd.to_datetime(df['obs']).dt.year
        else:
            df_sorted = df.sort_values(by='obs', ascending=False)
            df_sorted['Period'] = (df_sorted.index // 52) + 1
            df = df_sorted

        grouped_data = df.groupby('Period')

        if aggregation_type == 'sum':
            aggregated_data = grouped_data.sum(numeric_only=True)
        else:
            aggregated_data = grouped_data.mean(numeric_only=True)

        summary_df = aggregated_data
        summary_df['Period'] = summary_df.index

        start_col = (selected_page - 1) * columns_per_page
        end_col = min(selected_page * columns_per_page, num_columns)

        page_summary_data = summary_df.iloc[:, start_col:end_col]
        self.summary_data_model.update_data(page_summary_data.reset_index(drop=True))

    def update_raw_table(self):
        selected_page = self.raw_page_selector.value()

        start_col = (selected_page - 1) * columns_per_page
        end_col = min(selected_page * columns_per_page, num_columns)

        page_raw_data = self.df.iloc[:, start_col:end_col]
        self.raw_data_model.update_data(page_raw_data)

    def export_to_excel(self):
        options = QFileDialog.Options()
        file_path, _ = QFileDialog.getSaveFileName(self, "Save File", "", "Excel Files (*.xlsx)", options=options)
        if file_path:
            raw_data_df = self.raw_data_model._data
            summary_data_df = self.summary_data_model._data
            with pd.ExcelWriter(file_path) as writer:
                raw_data_df.to_excel(writer, sheet_name='Raw Data', index=False)
                summary_data_df.to_excel(writer, sheet_name='Data Summary', index=False)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    app.setStyleSheet("QWidget { background-color: #2b2b2b; }")  # Set background color to dark
    window = TwoTablesApp()
    window.show()
    sys.exit(app.exec())


SystemExit: 0

c:\Users\Magdalena\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
